In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error
import plotly.graph_objects as go
from itertools import product
from pmdarima import auto_arima
from uwv.config import KNMI_PROCESSED_DATA_DIR, KNMI_AVG_TEMP, CBS_OPENDATA_PROCESSED_DATA_DIR, CBS80072NED

2024-11-01 10:23:54.398 | INFO     | uwv.config:<module>:11 - PROJ_ROOT path is: C:\Users\c.hakker\OneDrive - VISTA college\Senior Stuff\Opleiding Data science\uwv


In [2]:
# Load the data
cbs = pd.read_parquet(CBS_OPENDATA_PROCESSED_DATA_DIR / f"{CBS80072NED}.parquet")
knmi = pd.read_parquet(KNMI_PROCESSED_DATA_DIR / f"{KNMI_AVG_TEMP}.parquet")

In [3]:
# Merge the datasets on 'period_year' and 'period_quarter_number'
cbsk = pd.merge(cbs, knmi, on=['period_year', 'period_quarter_number'], how="inner")

In [4]:
# Filter out rows where 'period_quarter_number' is 0
cbsk = cbsk[cbsk['period_quarter_number'] != 0]

In [5]:
# Map quarters to months and create the 'date' column
cbsk['month'] = cbsk['period_quarter_number'].map({1: 1, 2: 4, 3: 7, 4: 10})
cbsk['date'] = pd.to_datetime({'year': cbsk['period_year'], 'month': cbsk['month'], 'day': 1})

In [6]:
# Set this new 'date' column as the index
cbsk.set_index('date', inplace=True)

In [7]:
# Filter data based on 'sbi_title'
sbi_code = 'T001081'
filtered_cbs = cbsk[cbsk['sbi'] == sbi_code]

In [8]:
# Filter the data to include only dates from 2008 to 2023
filtered_cbs = filtered_cbs.loc['2008-01-01':'2023-12-31']

In [9]:
# Set the frequency to quarterly and drop any NaN values
filtered_cbs.index.freq = 'QS'
filtered_cbs = filtered_cbs.dropna()

In [10]:
# Ensure 'sick_leave_percentage' is in the correct format
filtered_cbs['sick_leave_percentage'] = filtered_cbs['sick_leave_percentage'].astype(float)

In [11]:
# Add rolling average and lagged exogenous variable
filtered_cbs['avg_temp_rolling'] = filtered_cbs['avg_temp'].rolling(window=3).mean()
filtered_cbs['avg_temp_rolling_lag1'] = filtered_cbs['avg_temp_rolling'].shift(1)
filtered_cbs = filtered_cbs.dropna(subset=['avg_temp_rolling', 'avg_temp_rolling_lag1', 'sick_leave_percentage'])

In [12]:
# Step 2: Define Training and Test Sets
train = filtered_cbs.loc[:'2021-12-31']
test = filtered_cbs.loc['2022-01-01':'2022-12-31']
forecast_period = pd.date_range(start='2023-01-01', periods=12, freq='QS')  # 12 periods for 3-year forecast

In [13]:
# Define the Function to Find the Best SARIMAX Model with Exogenous Variable
def find_best_sarimax(train, p_values, d_values, q_values, P_values, D_values, Q_values, m, exog_train, exog_test):
    best_aic = float("inf")
    best_order = None
    best_seasonal_order = None
    best_model = None

    for p, d, q in product(p_values, d_values, q_values):
        for P, D, Q in product(P_values, D_values, Q_values):
            try:
                model = SARIMAX(
                    train['sick_leave_percentage'],
                    order=(p, d, q),
                    seasonal_order=(P, D, Q, m),
                    exog=exog_train[['avg_temp_rolling', 'avg_temp_rolling_lag1']],
                    enforce_stationarity=False,
                    enforce_invertibility=False
                )
                results = model.fit(disp=False, method='powell', maxiter=3000)
                
                if results.aic < best_aic:
                    best_aic = results.aic
                    best_order = (p, d, q)
                    best_seasonal_order = (P, D, Q, m)
                    best_model = results
            except Exception as e:
                print(f"Error with order {(p, d, q)} and seasonal order {(P, D, Q, m)}: {e}")
                continue

    print(f"Best SARIMAX order: {best_order}, seasonal order: {best_seasonal_order}, AIC: {best_aic}")
    return best_model, best_order, best_seasonal_order, exog_test[['avg_temp_rolling', 'avg_temp_rolling_lag1']]

# %% 
# Initial broad grid search
p = range(0, 3)
d = range(0, 2)
q = range(0, 3)
P = range(0, 3)
D = range(0, 2)
Q = range(0, 3)
m = 4

best_model, best_order, best_seasonal_order, exog_test = find_best_sarimax(
    train, p, d, q, P, D, Q, m,
    exog_train=train[['avg_temp_rolling', 'avg_temp_rolling_lag1']],
    exog_test=test[['avg_temp_rolling', 'avg_temp_rolling_lag1']]
)

Best SARIMAX order: (0, 1, 1), seasonal order: (1, 1, 0, 4), AIC: -26.00884133815962


In [14]:
# Step 3: Make predictions using the best model
start = len(train)
end = start + len(test) - 1
predictions = best_model.predict(start=start, end=end, exog=exog_test).rename('SARIMAX Predictions')

In [15]:
# Step 5: Calculate MAE for 2022
mae_value = mean_absolute_error(test['sick_leave_percentage'], predictions)
print(f'MAE for all four quarters: {mae_value:.4f}')

# Calculate MAE for the next quarter (Q1)
mae_q1 = mean_absolute_error(test['sick_leave_percentage'].iloc[:1], predictions.iloc[:1])
print(f'MAE for the next quarter (Q1): {mae_q1:.4f}')

MAE for all four quarters: 0.2085
MAE for the next quarter (Q1): 0.4460


In [16]:
# Step 6: Extend exogenous data for 2022-2025 forecast (16 quarters total: 4 for test + 12 for forecast)
# Ensure we repeat the last available values to match the length
exog_forecast = pd.DataFrame({
    'avg_temp_rolling': list(exog_test['avg_temp_rolling']) + [exog_test['avg_temp_rolling'].iloc[-1]] * (16 - len(exog_test)),
    'avg_temp_rolling_lag1': list(exog_test['avg_temp_rolling_lag1']) + [exog_test['avg_temp_rolling_lag1'].iloc[-1]] * (16 - len(exog_test))
}, index=pd.date_range(start=test.index[0], periods=16, freq='QS'))

In [17]:
# Step 7: Forecast for 2022-2025 using the best model
start = len(train)
end = start + 16 - 1
forecast = best_model.predict(start=start, end=end, exog=exog_forecast).rename('SARIMAX Forecast')

In [18]:
# Combine test and forecast into one plot with Plotly
fig = go.Figure()

# Add actual sick leave percentage line (test data)
fig.add_trace(go.Scatter(
    x=test.index,
    y=test['sick_leave_percentage'],
    mode='lines+markers',
    name='Actual',
    line=dict(color='#0078d2', width=2)
))

# Add predictions line
fig.add_trace(go.Scatter(
    x=test.index,
    y=predictions,
    mode='lines+markers',
    name='Predictions',
    line=dict(color='orange', width=2, dash='dash')
))

# Add forecast line
fig.add_trace(go.Scatter(
    x=forecast_period,
    y=forecast,
    mode='lines+markers',
    name='Forecast',
    line=dict(color='green', width=2, dash='dot')
))

# Add MAE values as text annotations
fig.add_annotation(
    xref="paper", yref="paper", x=0.00, y=1.15, showarrow=False,
    text=f"MAE for all four quarters: {mae_value:.4f}",
    font=dict(size=12, color="black")
)

fig.add_annotation(
    xref="paper", yref="paper", x=0.00, y=1.20, showarrow=False,
    text=f"MAE for next quarter (Q1): {mae_q1:.4f}",
    font=dict(size=12, color="black")
)

# Set layout and design for better visualization
fig.update_layout(
    title='Sick Leave Percentage - Test, Predictions, and Forecast for Q Healthcare and Social Work',
    xaxis_title='Date',
    yaxis_title='Sick Leave Percentage',
    plot_bgcolor='white',
    xaxis=dict(
        showgrid=False,
        tickformat="%Y-%m",  # Year-Month format for clearer date labeling
        range=[test.index.min(), "2023-12-31"]  # Extend date range to 2025
    ),
    yaxis=dict(
        showgrid=True, gridcolor='lightgrey', showline=True, linewidth=0.5, linecolor='black'
    ),
    legend=dict(
        x=0.91,  # Move legend to the right of the plot
        y=1.5,
        traceorder="normal"
    ),
    font=dict(family="Roboto", size=14),
    margin=dict(l=50, r=50, t=80, b=50),
    width=1100, height=500
)

# Save the plot as a JPEG file
fig.write_image("Sick_leave_predict_sarimax.jpeg")

# Save the plot as an HTML file
fig.write_html("Sick_leave_predict_sarimax.html")

# Show plot
fig.show()
